In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

In [4]:
import psycopg2
import copy

from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [7]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

In [5]:
def get_def_info(word, from_lang, to_lang, multisence_dict, print_ouput = False):
    multisence_dict[word] = {'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                if print_ouput: print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                if print_ouput:print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    if print_ouput:print("ENG_DEFIN_EX", ex.text.strip())
                if print_ouput:print(current_definition)
                multisence_dict[word]['lemma_branch'].append(current_definition)
                current_definition = {}
                if print_ouput:print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_def_info("book", "english","russian",sence_dict)
sence_dict

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': []},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hotel room',
     "We've booked a trip to Spain for next month.",
     'Sorry, the hotel is fully booked (= has no more rooms).']},
   {'definition': 'to officially accuse someone of a crime',
    'local_word': 'заводить дело',
    'examples': ['Detectives booked him for resisting arrest.']},
   {'definition': 'If a sports official books you, they write 

In [8]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def nltk_lemmatize(word):
    pos = pos_tag(word_tokenize(word))[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    if wordnet_pos:
        lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
    else:
        lemma = lemmatizer.lemmatize(word)
    return lemma

nltk_lemmatize("baddest")

'baddest'

In [9]:
def get_wordform_info_from_reverso(eng_wordform, from_lang, to_lang, print_output = False):
    wordform_localsencewords_examples_dict = {}
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + eng_wordform 
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    first_string_passed = False
    translations_variants = []
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            if print_output:print(sence_word)
            wordform_localsencewords_examples_dict[sence_word] = []
            translations_variants.append(sence_word)
        first_string_passed = True
    time.sleep(random.uniform(0.4,1.2))
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        if print_output:
            print("========================================")
            print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + eng_wordform + "\w{0,1}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                wordform_localsencewords_examples_dict[transl_word].append(example)
                if print_output: print(example)
    return wordform_localsencewords_examples_dict
#w_locsence_ex_dict = get_wordform_info_from_reverso('book',"английский","русский", print_output = True )   
    

In [11]:
def merge_context_vs_oxford_examples(eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    oxford_sensewords_list = []
    oxford_lemmas_examples_pointers_dict = {}
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
        oxford_lemmas_examples_pointers_dict[definition_element['local_word']] = definition_element['examples']
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    for context_local_lemma in local_wordsence_examples_dict.keys():
        if print_output: print("\nNOWPROCESSING", context_local_lemma)
        orig_intersection_lemma = look_for_sublemma_word(context_local_lemma, oxford_sensewords_list)
        if not orig_intersection_lemma:
            if print_output: print(context_local_lemma, "\nno such lemma in orig dict")
            pass
        elif len(local_wordsence_examples_dict[context_local_lemma]) == 0:
            if print_output: print(context_local_lemma, "\ndoes not have examples with english wodform")
            pass
        else:
            #general_dict[eng_lemma]['lemma_branch'][context_local_lemma]['examples'].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print(oxford_lemmas_examples_pointers_dict[orig_intersection_lemma])
            oxford_lemmas_examples_pointers_dict[orig_intersection_lemma].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print("add",local_wordsence_examples_dict[context_local_lemma][:3] )
dct = copy.deepcopy(sence_dict)
#merge_context_vs_oxford_examples("book",w_locsence_ex_dict, dct, print_output = True)        
#dct

In [12]:
def look_for_sublemma_word(wordform, lemmas_list):
    if isinstance(lemmas_list, list) or isinstance(lemmas_list, set):
        for lemma in lemmas_list:
            if len(lemma.split()) == 1:
                if wordform in lemma or lemma in wordform:
                    #print(wordform)
                    return lemma
        return None
    elif isinstance(lemmas_list, str):
        if len(lemmas_list.split()) == 1:
            if wordform in lemmas_list or lemmas_list in wordform:
                return lemma
            else:
                return None
    else:
        return None #"no type" + str(type(lemmas_list))